In [133]:
import psycopg2
import pandas as pd

db_params = {'database' : 'postgres',
             'user' : 'postgres',
             'password' : 'admin',
             'host' : '127.0.0.1',
             'port' : '5432'}

conn = psycopg2.connect(database=db_params['database'], 
						user=db_params['user'],
                        password=db_params['password'], 
						host=db_params['host'],
                        port=db_params['port']
) 

cur = conn.cursor()

In [ ]:
cur.execute("ALTER TABLE branches ADD is_storage BOOLEAN DEFAULT FALSE;")
cur.execute("UPDATE branches SET is_storage=TRUE WHERE branch_name LIKE '%склад%' OR branch_name LIKE '%Склад%';")

conn.commit() 

In [90]:
cur.execute("ALTER TABLE branches ADD top_products VARCHAR[];")
conn.commit() 

1.1.  десять самых продаваемых товаров по складам;  
1.2.  десять самых продаваемых товаров по магазинам;  

Разделю все продажи на блоки по филиалам и сортирую по количеству продаж. Таким образом получу рейтинг самых продаваемых товаров для каждого филиала. Здесь же я исключил из выборки услуги доставки и обработки.  

В этом же запросе взял по 10 самых популярных товаров для каждого филиала и записал в поле таблицы филиалов с типом данных массив, его я создал ранее. 

In [91]:
cur.execute('''WITH RankedProducts AS (
  SELECT
    products.product_name,
    sales.sale_quantity,
    branches.branch_name,
    ROW_NUMBER() OVER (PARTITION BY branches.branch_key ORDER BY sales.sale_quantity DESC) AS ranking
  FROM
    sales
    JOIN products ON sales.product_key = products.product_key
    JOIN branches ON sales.branch_key = branches.branch_key
  WHERE 
      (products.product_name NOT LIKE '%Доставка%' AND products.product_name NOT LIKE '%доставка%' AND products.product_name NOT LIKE '%Обработка%')
)
UPDATE branches
SET top_products = (
  SELECT ARRAY_AGG(RankedProducts.product_name) AS top_products
  FROM RankedProducts
  WHERE RankedProducts.branch_name = branches.branch_name AND ranking BETWEEN 1 AND 10
)
WHERE EXISTS (
  SELECT 1 
  FROM RankedProducts
  WHERE RankedProducts.branch_name = branches.branch_name AND ranking BETWEEN 1 AND 10
);''')    

conn.commit() 

In [116]:
cur.execute('''SELECT branch_name, top_products FROM branches WHERE top_products IS NOT NULL''')
row = cur.fetchall()
display(pd.DataFrame(row))

,0,1
0,Ангарск 75 кв-л,[Память Secure Digital Memory Card 8 Gb (SDHC)...
1,Улан-Удэ Корпоратив,"[Кабель UTP, 4 пары, одножильный (solid), кат...."
2,яКиров Свободы,"[Разъем Noname [RJ45, 8P8C, кат. 5e, 1 шт], Ра..."
3,яКазань Московский,"[Кабель UTP, 4 пары, одножильный (solid), кат...."
4,Белгород ТЦ Пассаж,"[Футляр для 1 CD slim прозрачный, Память SanDi..."
...,...,...
273,Тула Корпоратив,"[Разъем Noname [RJ45, 8P8C, кат. 5e, 1 шт], Ба..."
274,яБлаговещенск Зейская,"[Кабель UTP, 4 пары, одножильный (solid), кат...."
275,Хаб Склад,"[Фонарь ЭРА K24 [кемпинговый фонарь, 24хLED, 3..."
276,Новосиб Склад,[Бумага офисная Xerox PerfectPrint (003R97759...


1.3.  десять первых магазинов по количеству продаж с суммой по количеству;  

Группирую филиалы и сортирую выборку по сумме проданых товаров в каждом филиале.

In [93]:
cur.execute('''SELECT
                branches.branch_name,
                SUM(sales.sale_quantity) AS total_quantity
            FROM
                sales
            JOIN branches ON branches.branch_key = sales.branch_key
            WHERE 
                branches.is_storage=False
            GROUP BY
                branches.branch_name
            ORDER BY 
                total_quantity DESC
            LIMIT 11''')

row = cur.fetchall()
display(pd.DataFrame(row[1:]))

,0,1
0,Новосиб Ватутина,94624.0
1,Ростов Красноармейская,79159.0
2,яВладивосток Луговая МБТ,74879.0
3,Омск на Гагарина,70353.0
4,яКомсомольск Аллея Труда,66382.0
5,ЕКБ Вайнера,63229.0
6,яН.Новгород Пл. Революции,63049.0
7,яСаратов Университетская,60728.0
8,Ростов Вавилония,60458.0
9,Краснодар ТЦ Кавказ,60359.0


1.4.  рейтинг товаров согласно суммарному количеству проданного товара за всю историю наблюдений со средним количеством продаж за день по убыванию;  

Здесь использую не EXTRACT, а DATE_TRUNC, потому что мне нужно обрезать TAMESTAMP до дней, в то время, как EXTRACT возвращает мне необходимую отметку из TIMESTAMP.  

Здесь я считаю количество проданных товаров за день, сумму каждого из дней, затем вычисляю из этого среднее количество продаж в день.

In [135]:
cur.execute('''WITH ProductRanking AS (
            SELECT
                products.product_key,
                products.product_name,
                SUM(sales.sale_quantity) AS total_quantity,
                COUNT(DISTINCT DATE_TRUNC('day', sales.sale_period)) AS days_count,
                SUM(sales.sale_quantity) / COUNT(DISTINCT DATE_TRUNC('day', sales.sale_period)) AS avg_sales,
                ROW_NUMBER() OVER (ORDER BY SUM(sales.sale_quantity) DESC) AS ranking
            FROM
                sales
                JOIN products ON sales.product_key = products.product_key
            GROUP BY
                products.product_key
            )
            SELECT
                product_name,
                total_quantity,
                avg_sales
            FROM
                ProductRanking
            ORDER BY
                ranking;
            ''')

row = cur.fetchall()
display(pd.DataFrame(row))

,0,1,2
0,Доставка внутри региона,1036108.0,9088.666667
1,"Разъем Noname [RJ45, 8P8C, кат. 5e, 1 шт]",256369.0,2136.408333
2,"Кабель UTP, 4 пары, одножильный (solid), кат. ...",212994.0,1774.950000
3,Обработка грузов на РРЦ,171546.0,3500.938776
4,Доставка от поставщика,43338.0,498.137931
...,...,...,...
19841,Сотовый телефон Samsung GT-S5330 WAVE 533 Bla...,-2.0,-0.500000
19842,"Монитор LG 21.5"" Flatron W2246S [1920x1080, D...",-2.0,-1.000000
19843,"Монитор LG LCD 20"" Flatron W2046S [1600x900, D...",-2.0,-1.000000
19844,"Монитор LG LCD 19"" Flatron W1934S/S-PF [1440x9...",-3.0,-1.000000


1.5.  Два лучших филиала согласно суммарному количеству проданного товара за всю историю наблюдений в регионе Урал по городу Екатеринбург с суммой продаж за январь  

Узнал код города Екатеринбург, сгруппировал продажи по филиалам, выделил продажи в Екатеринбурге в январе.  

Вывел два наибольших значения, предварительно отсортировав их по убыванию.

In [105]:
cur.execute('''SELECT
                cities.city_name,
                cities.city_key
            FROM
                cities
            WHERE 
                cities.city_name = 'Екатеринбург';''')

row = cur.fetchall()
display(pd.DataFrame(row))

,0,1
0,Екатеринбург,83878977-f329-11dd-9648-00151716f9f5


In [111]:
cur.execute('''SELECT
                branches.branch_name,
                SUM(sales.sale_quantity) AS total_quantity
            FROM
                sales
            JOIN branches ON branches.branch_key = sales.branch_key
            WHERE 
                branches.city_key = '83878977-f329-11dd-9648-00151716f9f5' AND
                EXTRACT(MONTH FROM sale_period) = 1
            GROUP BY
                branches.branch_name
            ORDER BY 
                total_quantity DESC
            LIMIT 2''')

row = cur.fetchall()
display(pd.DataFrame(row))

,0,1
0,ЕКБ старый cклад,42489.0
1,ЕКБ Вайнера,15605.0


1.6.  Требуется рассчитать и вывести в какие часы и в какой день недели происходит максимальное количество продаж.  

Для того, чтобы выделить день недели и часы из TIMESTAMP, воспользуюсь функцией EXTRACT, отсортировав данные по убыванию по количеству продаж.  

В результате получаю период с 13:00 до 15:00 в воскресенье. Здесь я взял три часа времени. 

In [94]:
cur.execute('''SELECT
  EXTRACT(DOW FROM sale_period) AS day_of_week,
  EXTRACT(HOUR FROM sale_period) AS hour,
  COUNT(*) AS sales_count
FROM
  sales
GROUP BY
  day_of_week, hour
ORDER BY
  sales_count DESC
LIMIT 3;''')

row = cur.fetchall()
display(pd.DataFrame(row))

,0,1,2
0,6,14,153267
1,6,15,150048
2,6,13,147808


In [ ]:
cur.close()
conn.close()